<p style="text-align:center; font-size:larger; color: #40E0D0;"><strong>Duales en Python y Cython</strong></p>

**Una clase llamada duales que soporte las operaciones de suma, resta, multiplicación, división, cos, sin, tan, ln, exp.**

**Esta clase debe comportarse como los números duales y proporcionar en la parte real del dato el valor de la operación, y en la parte dual el valor de la derivada.**

La aplicación de los numeros duales como una diferenciación automática es fascinante. 

Si consideramos los numeros duales de la forma $a + b\epsilon$ y los empleamos para evaluar la función sobre los mismos, se puede usar esta relación para calculas las derivadas a prácticamente cualquier función real.

En términos generales, se puede extender cualquier función real (analítica) a los numeros duales haciendo su desarrollo en serie de taylor:

${\displaystyle f(a+b\varepsilon )=\sum _{n=0}^{\infty }{\frac {f^{(n)}(a)b^{n}\varepsilon ^{n}}{n!}}=f(a)+bf'(a)\varepsilon ,}$

donde se ha tenido en cuenta que por definición, $\epsilon^2 = 0$. 

Se puede ver como se ha calculado automáticamente la derivada de la composición.

<p style="text-align:center; font-size:larger;"><strong> Python </strong></p>

In [6]:
# Importamos la librería math
import math
from typing import Any

# Definimos la clase duales
class Duales:
    def __init__(self, parte_real, parte_dual): 
        self.parte_real = parte_real
        self.parte_dual = parte_dual
        
    def __add__(self,other):
        return Duales(self.parte_real + other.parte_real, self.parte_dual + other.parte_dual)
    
    def __sub__(self,other):
        return Duales(self.parte_real - other.parte_real, self.parte_dual - other.parte_dual)
    
    def __mul__(self,other):
        return Duales(self.parte_real * other.parte_real, (self.parte_real * other.parte_dual + self.parte_dual * other.parte_real))
    
    def div(self,other):
        if (other.parte_real != 0):
            return Duales(self.parte_real / other.parte_real, (self.parte_dual * other.parte_real - self.parte_real * other.parte_dual) / (other.parte_real ** 2))
        else:
            if (other.parte_dual != 0 and self.parte_real != 0):
                return "No tiene solución."
            if (other.parte_dual != 0 and self.parte_real == 0):
                return "La división no está definida para números duales puramente no reales."
                    
    def cos(self):
        return Duales(math.cos(self.parte_real), -math.sin(self.parte_real) * self.parte_dual)

    def sin(self):
        return Duales(math.sin(self.parte_real), math.cos(self.parte_real) * self.parte_dual)

    def tan(self):
        if math.cos(self.parte_real) == 0:
            return "Error al calcular la tangente: cos(a)=0"
        else:
            return Duales(math.tan(self.parte_real), self.parte_dual / (math.cos(self.parte_real) ** 2))
    
    def log(self):
        if self.parte_real <= 0:
            return "Error al calcular el logaritmo: No está definido para a<=0"
        else:
            return Duales(math.log(self.parte_real), self.parte_dual / self.parte_real)  
    
    def exp(self):
        return Duales(math.exp(self.parte_real), self.parte_dual * math.exp(self.parte_real))
          
    def __repr__(self):
        return f"{self.parte_real} + {self.parte_dual} ε"

In [7]:
#%%timeit

# Números duales (a + bε) -> Duales(parte_real, parte_dual)
dual1 = Duales(2,4) 
dual2 = Duales(3,1) 

# Operaciones Algebraicas básicas:
suma  = dual1 + dual2
resta = dual1 - dual2
producto = dual1 * dual2
cociente = dual1.div(dual2)

# Operaciones trigonométricas:
cos1 = dual1.cos()
cos2 = dual2.cos()
sen1 = dual1.sin()
sen2 = dual2.sin()
tan1 = dual1.tan()
tan2 = dual2.tan()

# Operaciones logarítmicas y exponenciales:
log1 = dual1.log()
log2 = dual2.log()
exp1 = dual1.exp()
exp2 = dual2.exp()

print("Primer número dual (dual1):\t",  dual1)
print("Segundo número dual (dual2):\t", dual2)
print("Suma números duales:\t", suma)
print("Resta números duales:\t", resta)
print("Producto números duales:\t", producto)
print("División números duales:\t", cociente)
print("Coseno dual1:\t", cos1)
print("Coseno dual2:\t", cos2)
print("Seno dual1:\t", sen1)
print("Seno dual2:\t", sen2)
print("Tangente dual1:\t", tan1)
print("Tangente dual2:\t", tan2)
print("Logaritmo dual1:\t", log1)
print("Logaritmo dual2:\t", log2)
print("Exponencial dual1:\t", exp1)
print("Exponencial dual2:\t", exp2)

Primer número dual (dual1):	 2 + 4 ε
Segundo número dual (dual2):	 3 + 1 ε
Suma números duales:	 5 + 5 ε
Resta números duales:	 -1 + 3 ε
Producto números duales:	 6 + 14 ε
División números duales:	 0.6666666666666666 + 1.1111111111111112 ε
Coseno dual1:	 -0.4161468365471424 + -3.637189707302727 ε
Coseno dual2:	 -0.9899924966004454 + -0.1411200080598672 ε
Seno dual1:	 0.9092974268256817 + -1.6645873461885696 ε
Seno dual2:	 0.1411200080598672 + -0.9899924966004454 ε
Tangente dual1:	 -2.185039863261519 + 23.09759681616767 ε
Tangente dual2:	 -0.1425465430742778 + 1.020319516942427 ε
Logaritmo dual1:	 0.6931471805599453 + 2.0 ε
Logaritmo dual2:	 1.0986122886681098 + 0.3333333333333333 ε
Exponencial dual1:	 7.38905609893065 + 29.5562243957226 ε
Exponencial dual2:	 20.085536923187668 + 20.085536923187668 ε


<p style="text-align:center; font-size:larger;"><strong> Cython </strong></p>

In [8]:
# Cargamos la extensión de Cython
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [9]:
%%cython -a

from libc.math cimport cos, sin, exp, log, tan

cdef class DualesCython:
    cdef double _parte_real
    cdef double _parte_dual
    
    def __init__(self, double parte_real, double parte_dual): 
        self._parte_real = parte_real
        self._parte_dual = parte_dual
    
    def get_parte_real(self):
        return self._parte_real
    
    def get_parte_dual(self):
        return self._parte_dual
    
    def __add__(self, DualesCython other):
        return DualesCython(self._parte_real + other._parte_real, self._parte_dual + other._parte_dual)

    def __sub__(self, DualesCython other):
         return DualesCython(self._parte_real - other._parte_real, self._parte_dual - other._parte_dual)

    def __mul__(self, DualesCython other):
        return DualesCython(self._parte_real * other._parte_real, (self._parte_real * other._parte_dual + self._parte_dual * other._parte_real))

    def div(self,other):
        if (other.parte_real != 0):
            return DualesCython(self.parte_real / other.parte_real, (self.parte_dual * other.parte_real - self.parte_real * other.parte_dual) / (other.parte_real ** 2))
        else:
            if (other.parte_dual != 0 and self.parte_real != 0):
                return "No tiene solución"
            if (other.parte_dual != 0 and self.parte_real == 0):
                return "La división no está definida para números duales puramente no reales"

    def coseno(self):
        return DualesCython(cos(self.parte_real), -sin(self.parte_real) * self.parte_dual)

    def seno(self):
        return DualesCython(sin(self.parte_real), cos(self.parte_real) * self.parte_dual)

    def tangente(self):
        if cos(self.parte_real) == 0:
            return "Error al calcular la tangente: cos(a)=0"
        else:
            return DualesCython(tan(self.parte_real), self.parte_dual / (cos(self.parte_real) ** 2))

    def logaritmo(self):
        if self.parte_real <= 0:
            return "Error al calcular el logaritmo: No está definido para a<=0 "
        else:
            return DualesCython(log(self.parte_real), self.parte_dual / self.parte_real) 

    def exponencial(self):
        return DualesCython(exp(self.parte_real), self.parte_dual * exp(self.parte_real))

    def __repr__(self):
            return f"{self._parte_real} + {self._parte_dual} ε"

    property parte_real:
        def __get__(self):
            return self.get_parte_real()
    
    property parte_dual:
        def __get__(self):
            return self.get_parte_dual()

In [10]:
#%%timeit
d1 = DualesCython(2,4)
d2 = DualesCython(3,1)
d3 = d1 + d2
d4 = d1 - d2
d5 = d1 * d2
d6 = d1.div(d2)
d7 = d1.coseno()
d8 = d2.coseno()
d9 = d1.seno()
d10 = d2.seno()
d11 = d1.tangente()
d12 = d2.tangente()
d13 = d1.logaritmo()
d14 = d2.logaritmo()
d15 = d1.exponencial()
d16 = d2.exponencial()

print("Primer número dual (dual1):\t", d1)
print("Segundo número dual (dual2):\t", d2)
print("Suma números duales:\t", d3)
print("Resta números duales:\t", d4)
print("Producto números duales:\t", d5)
print("División números duales:\t", d6)
print("Coseno dual1:\t", d7)
print("Coseno dual2:\t", d8)
print("Seno dual1:\t", d9)
print("Seno dual2:\t", d10)
print("Tangente dual1:\t", d11)
print("Tangente dual2:\t", d12)
print("Logaritmo dual1:\t", d13)
print("Logaritmo dual2:\t", d14)
print("Exponencial dual1:\t", d15)
print("Exponencial dual2:\t", d16)

Primer número dual (dual1):	 2.0 + 4.0 ε
Segundo número dual (dual2):	 3.0 + 1.0 ε
Suma números duales:	 5.0 + 5.0 ε
Resta números duales:	 -1.0 + 3.0 ε
Producto números duales:	 6.0 + 14.0 ε
División números duales:	 0.6666666666666666 + 1.1111111111111112 ε
Coseno dual1:	 -0.4161468365471424 + -3.637189707302727 ε
Coseno dual2:	 -0.9899924966004454 + -0.1411200080598672 ε
Seno dual1:	 0.9092974268256817 + -1.6645873461885696 ε
Seno dual2:	 0.1411200080598672 + -0.9899924966004454 ε
Tangente dual1:	 -2.185039863261519 + 23.09759681616767 ε
Tangente dual2:	 -0.1425465430742778 + 1.020319516942427 ε
Logaritmo dual1:	 0.6931471805599453 + 2.0 ε
Logaritmo dual2:	 1.0986122886681098 + 0.3333333333333333 ε
Exponencial dual1:	 7.38905609893065 + 29.5562243957226 ε
Exponencial dual2:	 20.085536923187668 + 20.085536923187668 ε


<p style="text-align:center; font-size:larger;"><strong> Resultados </strong></p>

Runtime Python: 89.1 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Runtime Cython: 78.6 µs ± 6.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Los resultados que se obtienen no son muy significativos. Sin embargo, se confirma la hipótesis inicial y se demuestra que programar en Cython es más eficiente en términos de rendimiento que Python.